1. 데이터 읽기 - 데이터 읽고 dataframe형태로 저장

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# dataframe형태로 불러오기
data = pd.read_csv('data/uci-secom.csv')

In [ ]:
data.head()

In [ ]:
data.info()
data.shape

In [ ]:
# 수치형 변수의 데이터 정보를 요약하여 출력
data.describe()

2. 데이터 정제 - 결측값 및 이상치 처리

In [ ]:
# isnull()은 결측값이 있는지 True, False로 반환
data.isnull().sum()

In [ ]:
data = data.replace(np.NaN, 0)
data.isnull().sum()

In [ ]:
# axis = 0은 행방향 동작
data = data.drop(columns = ['Time'], axis = 1)
data.shape

In [ ]:
data

3. 데이터 시각화

3.1 Pass/Fail 시각화

In [ ]:
# 막대그래프 / value_counts() : 합계
data['Pass/Fail'].value_counts().plot(kind='bar')
# 도수분포표
data['Pass/Fail'].value_counts()

3.2 센서 데이터 시각화 하기

In [ ]:
# seaborn : 다수의 feature 데이터를 한눈에 보는 라이브러리
# pairplot() : 컬럼끼리 비교
data_test = data[['3', '4', '5', 'Pass/Fail']]
data_test

In [ ]:
sns.pairplot(data_test)

In [ ]:
# vars : 특정한 컬럼끼리 비교
sns.pairplot(data_test, height = 5, vars = ['3', '4'])

3.3 59번 센서 시각화

In [ ]:
# subplots : 한 번에 여러 그래프를 보여준다
# ax : 각각의 그래프
# fig(가로, 세로) : 크기
fig, ax = plt.subplots(figsize = (8,6))
sns.set(style = 'darkgrid')
sns.distplot(data['59'], color = 'darkblue')
plt.title('59 Sensor Measurements', fontsize = 20)

In [ ]:
plt.rcParams['figure.figsize'] = (10, 16)
# subplots(행, 열, 인덱스)
plt.subplot(3, 1, 1)
sns.distplot(data['59'], color = 'darkblue')
plt.title('59 Sensor Measurements', fontsize = 20)

plt.subplot(3, 1, 2)
sns.distplot(data[data['Pass/Fail']==1]['59'], color = 'darkgreen')
plt.title('59 Sensor Measurements', fontsize = 20)

plt.subplot(3, 1, 3)
sns.distplot(data[data['Pass/Fail']==-1]['59'], color = 'red')
plt.title('59 Sensor Measurements', fontsize = 20)

In [ ]:
plt.rcParams['figure.figsize'] = (15, 10)
# 나누었던 그래프 한번에 출력
sns.distplot(data['59'], color = 'darkblue')
sns.distplot(data[data['Pass/Fail']==1]['59'], color = 'darkgreen')
sns.distplot(data[data['Pass/Fail']==-1]['59'], color = 'red')

plt.title('59 Sensor Measurements', fontsize = 20)

4. 데이터 전 처리
 - 전체 데이터를 feature 데이터인 x 와 label 데이터인 y로 분리
 - StandardScaler를 통한 데이터 표준화

4.1 x와 y로 분리

In [ ]:
x = data.drop(columns = ['Pass/Fail'], axis = 1)
y = data['Pass/Fail']
# ravel : 다차원을 1차원으로 풀기
y = y.to_numpy().ravel() 
y

In [ ]:
type(y)

In [ ]:
data_test = pd.read_csv("data/uci-secom-test.csv")
x_test = data_test.drop(columns = ['Pass/Fail'], axis = 1)
y_test = data_test['Pass/Fail'].to_numpy().ravel() 

In [ ]:
x_test

4.2 데이터 표준화 
- 각 변수 마다의 스케일 차이를 맞추기 위하여 표준화를 수행
- 데이터의 피처 각각이 평균이 0, 분산이 1인 가우시안 정규분포 형태

In [ ]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
x_train = sc.fit_transform(x)
x_test = sc.transform(x_test)
y_train = y

In [ ]:
x_train_sc = pd.DataFrame(data=x_train)
print("평균")
print(x_train_sc.mean())
print("분산")
print(x_train_sc.var())

In [ ]:
x_train_sc

5. 머신러닝 모델 학습 - score를 통해 accuracy 출력

로지스틱 회귀
 - 선형 회귀 방식을 분류에 적용한 알고리즘
 - 시그모이드 함수 최적선을 찾고 반환 값을 확률로 간주해 확률에 따라 분류
 - 0과 1로 분류

5.1 기본 분류 모델 학습 - 로지스틱 분류기

In [ ]:
from sklearn.linear_model import LogisticRegression
# max_iter : 반복횟수를 정하는 파라미터
model = LogisticRegression(max_iter = 5000)
model.fit(x_train, y_train)
print(model.score(x_train, y_train))
print(model.score(x_test, y_test))

In [ ]:
# 중요도 계산 / 가중치 값들의 크기로 판단하기에 .coef로 해당 값들을 불러온다
abs_coef = np.abs(model.coef_).ravel()
abs_coef

In [ ]:
#상위 20개 출력
LR_import_x = [str(i[0]) for i in sorted(enumerate(abs_coef), key=lambda x:x[1], reverse=True)]

plt.bar(LR_import_x[:20], sorted(abs_coef, reverse=True)[:20])

plt.rcParams['figure.figsize'] = (15, 10)
plt.xlabel('Features')
plt.ylabel('Weight absolute values')
plt.show()

5.2 다양한 분류 모델 학습

In [ ]:
# 분류 모델 중 XGB모델 있으니 참조

from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
import xgboost as xgb


#여러 모델을 append해서 추가합니다. 
models = []
models.append(('LDA', LinearDiscriminantAnalysis()))  # LDA 모델
models.append(('KNN', KNeighborsClassifier()))  # KNN 모델
models.append(('CART', DecisionTreeClassifier()))  # 의사결정트리 모델
models.append(('NB', GaussianNB()))  # 가우시안 나이브 베이즈 모델
models.append(('RF', RandomForestClassifier()))  # 랜덤포레스트 모델
models.append(('SVM', SVC(gamma='auto')))  # SVM 모델

for name, model in models:
    # fit으로 학습을 합니다. 
    model.fit(x_train, y_train)
    
    # score 함수를 사용하여 모델의 성능을 확인합니다.
    msg = "%s - train_score : %f, test score : %f" % (name, model.score(x_train, y_train), model.score(x_test, y_test))
    print(msg)

6. 평가 및 예측
 - recall : 예측한 이상 있음 대비 실제 이상 있음의 비율 (accuracy에서 놓칠 수 있는 결과 해석 보충)
  - 정밀도(Precision)
   - 재현율 (recall)

6.1 Condusion Matrix

In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns 

# LinearDiscriminantAnalysis 모델의 confusion matrix를 사용하기 위하여 학습용 데이터의 예측값을 저장합니다.
# models[0]는 LDA
model_predition_train = models[0][1].predict(x_train)

cm_train = confusion_matrix(y_train, model_predition_train)

plt.rcParams['figure.figsize'] = (5, 5)
sns.set(style = 'dark', font_scale = 1.4)

# annot은 annotate each cell with numeric value로 셀에 숫자값을 표시하는지 정하는 것입니다. 
# cmap으로 색깔을 지정할 수 있습니다. cmap='RdYlGn_r' cmap="YlGnBu"
ax = sns.heatmap(cm_train, annot=True)
plt.xlabel('Prediction')
plt.ylabel('Real Data')
ax.set_xticklabels((-1,1))
ax.set_yticklabels((-1,1))
plt.show()
cm_train

6.2 Precision & Recall

In [ ]:
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score

print("Recall score: {}".format(recall_score(y_test, models[0][1].predict(x_test))))
print("Precision score: {}".format(precision_score(y_test, models[0][1].predict(x_test))))

6.3 테스트 데이터의 예측값 출력

In [ ]:
for i in range(10): 
    
    # LDA 모델을 사용하였습니다.
    # reshape()에서 -1이 들어간 곳은 가변적으로 바꿉니다. 예를 들어 12개의 원소가 있고 reshape(-1,2)를 하면 열 2개를 맞추기 위해서 행을 6개로 바꿉니다. 
    prediction = models[0][1].predict(x_test[i].reshape(1,-1))
    
    print("{} 번째 테스트 데이터의 예측 결과: {}, 실제 데이터: {}".format(i, prediction[0], y_test[i]))